In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.common.exceptions import WebDriverException, TimeoutException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd, numpy as np
from pandas.io.formats.excel import ExcelFormatter
import os
import nbimporter
from datetime import datetime
#from pipeline import raw_cleaner
#from pipeline import idx_find
#from pipeline import find_date_format
#from pipeline import find_lig
#from pipeline import fn_acc_record
import warnings
import re
warnings.filterwarnings('ignore')
from openpyxl import load_workbook

In [38]:
# Geckodriver yolu
geckodriver_path = "../set/geckodriver"

# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)


# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)
# Hedef web sayfasına git

macKolikUrl="https://www.mackolik.com/futbol/canli-sonuclar"
driver.get(macKolikUrl)

#time.sleep(5)
wait = WebDriverWait(driver, 30)
cookies=wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#didomi-notice-agree-button')))
cookies.click()

#click for ordering by matches start time
#clock=wait.until(EC.element_to_be_clickable((By.XPATH, "//span[.='Saat']")))
#clock.click()

dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))

# Click on the element
dateIcon.click()
butunTakvim = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.widget-datepicker')))
html_contentTakvim = butunTakvim.get_attribute('outerHTML')
# print(html_contentTakvim)



# BeautifulSoup ile HTML içeriğini parse et
soup = BeautifulSoup(html_contentTakvim, 'html.parser')

# Günü ve Yılı al
day = int(soup.find('td', {'class': 'widget-datepicker__calendar-body-cell--selected-day'}).text.strip())
year = int(soup.find('div', {'class': 'widget-datepicker__selector--year'}).find('div', {'class': 'widget-datepicker__value'}).text.strip())

#print(f"day: {day}, year: {year}")

current_year = int(driver.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__value').text)
previous_year_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__nav.widget-datepicker__nav--previous")))

# Hedef yılı belirle (örneğin, 2020)
target_year = 2023

# Hedef yıla kadar önceki yıl düğmesine tıkla
while current_year > target_year:
    #print(str(target_year)+ "target year")
    previous_year_button.click()
    #time.sleep(1)
    current_year -= 1
    #print(current_year)
    # Yeni yılı kontrol et
    current_year = int(driver.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__value').text)
    #print(current_year)

    
# calc of months
previous_month_button= wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__nav.widget-datepicker__nav--previous")))
#wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
current_month = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__value")))
month_str = current_month.text.strip()[:3]

#print(month_str)
# Map the month abbreviation to a numeric value
month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
#print(month_str+"month_str")
month_numeric = month_mapping.get(month_str)

target_month=12
month_numeric = month_mapping.get(month_str)


# Loop until you reach the target month
while month_numeric != target_month:
    #print(str(month_numeric)+"month_numeric")
    #print(str(target_month)+"target_month")

    
    previous_month_button.click()
    #time.sleep(1)
    
    current_month = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__value")))
    month_str = current_month.text.strip()[:3]
    #print(month_str+"month_str")
    month_numeric = month_mapping.get(month_str)
    #print(month_numeric)
    #print(target_month)
    
target_day=18
day_element = driver.find_element(By.XPATH, f"//td[contains(@class, 'widget-datepicker__calendar-body-cell') and text()='{target_day}']")
#time.sleep(19)                   #//td[contains(@class, 'widget-datepicker__calendar-body-cell') and text()='{target_day}']
day_element.click()

time.sleep(5)


# wait for page loaded
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".page-livescore-index__container"))
)

#daily all  matches rows container elements xpath
container_css = "div.match-row__match-content"  #(//div[@class='match-row__match-content'])[1]


wait_time = 15

#daily all  matches rows container ELEMENT
# allLinksElements = WebDriverWait(driver, wait_time).until(
#     EC.visibility_of_all_elements_located((By.CSS_SELECTOR, container_css))
# )

# Container elementini bul (alternative)
#allLinksElements = driver.find_elements(By.XPATH, container_xpath)

#dataframe all bettings odds one match
#df_all = pd.DataFrame()
count = 0
iteration_count = 0
max_iterations = 2

while True:
    try:
        # Container elementini bul
        allLinksElements = WebDriverWait(driver, wait_time).until(
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, container_css))
        )

       
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=34):
            result=[]
            e = None 
            try:
                print(f"(try Worked... {str(i)})")
                # Precautionary for page loading
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))
                #driver.implicitly_wait(10)
                
                #element.click()
                max_retry_count = 5
                retry_delay = 5  # saniye cinsinden

                for _ in range(max_retry_count):
                    try:
                        elementx.click()
                        break  # If the click is successful, exit the loop
                    except (WebDriverException, TimeoutException, BadGatewayException) as e:
                        if "502 Bad Gateway" in str(e):
                            print("502 Bad Gateway error. Retrying...")
                            break
                        print(f"Error occurred: {e}")
                        time.sleep(retry_delay)

                if "502 Bad Gateway" in str(e):
                    # Close the current tab and switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue
                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                
                # Perform actions in the new tab
                iddiaButton = WebDriverWait(driver, wait_time).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                )

                # get class attribute values...
                tx = iddiaButton.get_attribute('class')

                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    # Top scoreboard table element
                    macScorboard = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.p0c-soccer-match-details-header"))
                    )
                    
                    # Main betting table element
                    macDetailsData = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, ".widget-iddaa-markets__markets-list"))
                    )
                    
                    # Scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    for line_sb in lines:
                        result += [line_sb]
                    print(f'-------------------\n{result}')

                    
                    # Get match details as string text
                    mac_Details_Data = macDetailsData.text
                    detail_lines = mac_Details_Data.strip().split('\n')## detail_lines is a list
                    
                    for line_md in detail_lines:
                        result+=[line_md]
                        
                    #print(result)
                    
                    crude_df=pd.DataFrame(result, columns=[0])
                    #pd.options.display.max_rows = None
                    #print(crude_df)
                    
                    
                    # directory = './dataSet'
                    # if not os.path.exists(directory):
                    #     os.makedirs(directory)

                    # DataFrame'i Excel dosyasına kaydet
                    crude_df.to_excel(f'../data/new_data_raw_{i}_.xlsx')
                    cleaned_df=raw_cleaner(list(crude_df[0]))
                    fn_acc_record('../data/df_all_cleaned_games.xlsx', cleaned_df, header=False, index=False)
                    
                    
                    
                    
                    # Print match details...
                    #print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

            # Increment the iteration count
#             iteration_count += 1

#             # Check if it's time to restart the loop
#             if iteration_count % max_iterations == 0:
#                 print(f"Restarting the loop after {max_iterations} iterations.")
#                 restart_loop=True
#                 break
                
                
#         if restart_loop:
#             break       

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:630:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1499:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [39]:
def fn_acc_record(filename, df, sheet_name='records', startrow=None, **to_excel_kwargs):
    try:
        # ExcelWriter ile dosyayı aç
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # Eğer belirtilen sayfa varsa, startrow'u son satır olarak ayarla
            if sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row
            else:
                # Belirtilen sayfa yoksa, startrow'u 0 olarak başlat
                startrow = 0

            # DataFrame'i yazdır
            df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, **to_excel_kwargs)

    except FileNotFoundError:
        # Dosya yoksa, yeni bir dosya oluştur ve startrow'u 0 olarak başlat
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            startrow = 0
            df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, **to_excel_kwargs)

In [40]:
def idx_find(L, text,jump):
    for idx, value in enumerate(L):
        if text in str(value):
            get_idx = L.index(value)
            return get_idx + jump
    return -1

In [41]:
def find_date_format(L):
    # Veriyi dolaş, uygun tarihi bul ve döndür
    for data in L:
        date_dash = re.search(r'\d{2}-\d{2}-\d{2}', data)
        date_slash = re.search(r'\d{2}/\d{2}/\d{2}', data)
        
        if date_dash:
            return date_dash.group()
        elif date_slash:
            return date_slash.group()

    return None

In [42]:
def find_IYScore(L):
    for data in L:
        if isinstance(data, str):
            match = re.search(r'\(İY (\d+) - (\d+)\)', data)
            if match:
                return match.group(1) + '-' + match.group(2)
    return '-'

In [43]:
def find_lig(L, patern):
    for data in L:
        lig = re.search(patern, data)
        if lig:
            return lig.group(1).strip()
    return None

In [44]:
#renewed
def raw_cleaner(L):
    # if not any data, assign nan value
    L.append(np.nan) 
    IlkYariSonucu=''
    MacSonucu=''
    
    
    if any("ERT" in str(item) for item in L):
        MacSonucu = "ERT"
        IlkYariSonucu = '-'
    elif any("components" in str(item) for item in L):
        IlkYariSonucu = '-'
        MacSonucu = '-'
    else:
        IlkYariSonucu = find_IYScore(L)
        MacSonucu = f"{L[2]}-{L[4]}"
        
       
    
    cleaned = {
        #'MacTarihi':L[idx_find(L, 'Hafta',0)].split('|')[0],
  
        'MacTarihi':find_date_format(L),
        
        'Lig':find_lig(L, r'\|\s*(.*?)\s*\|'),
        
        'EvSahibi':L[0],
        'KonukEkip':L[1],
        'IlkYariSonucu': IlkYariSonucu,
        'MacSonucu': MacSonucu,
        
        'Ms1': L[idx_find(L, 'MAÇ SONUCU',2)],
        'Ms0':L[idx_find(L, 'MAÇ SONUCU',4)],
        'Ms2':L[idx_find(L, 'MAÇ SONUCU',6)],
        
        'CS10':L[idx_find(L, 'ÇIFTE ŞANS',2)],
        'CS12':L[idx_find(L, 'ÇIFTE ŞANS',4)],
        'CS02':L[idx_find(L, 'ÇIFTE ŞANS',6)],
        
        'H10Ms1':L[idx_find(L, 'HND. MS (1:0)',2)],
        'H10Ms0':L[idx_find(L, 'HND. MS (1:0)',4)],
        'H10Ms2':L[idx_find(L, 'HND. MS (1:0)',6)],
        'H20Ms1':L[idx_find(L, 'HND. MS (2:0)',2)],
        'H20Ms0':L[idx_find(L, 'HND. MS (2:0)',4)],
        'H20Ms2':L[idx_find(L, 'HND. MS (2:0)',6)],
        'H30Ms1':L[idx_find(L, 'HND. MS (3:0)',2)],
        'H30Ms0':L[idx_find(L, 'HND. MS (3:0)',4)],
        'H30Ms2':L[idx_find(L, 'HND. MS (3:0)',6)],
        'H01Ms1':L[idx_find(L, 'HND. MS (0:1)',2)],
        'H01Ms0':L[idx_find(L, 'HND. MS (0:1)',4)],
        'H01Ms2':L[idx_find(L, 'HND. MS (0:1)',6)],
        'H02Ms1':L[idx_find(L, 'HND. MS (0:2)',2)],
        'H02Ms0':L[idx_find(L, 'HND. MS (0:2)',4)],
        'H02Ms2':L[idx_find(L, 'HND. MS (0:2)',6)],
        'H03Ms1':L[idx_find(L, 'HND. MS (0:3)',2)],
        'H03Ms0':L[idx_find(L, 'HND. MS (0:3)',4)],
        'H03Ms2':L[idx_find(L, 'HND. MS (0:3)',6)],
        
        'Ms1den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',2)],
        'Ms0den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',4)],
        'Ms2den1':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',6)],       
        'Ms1den0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',8)],
        'Ms0dan0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',10)],
        'Ms2den0':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',12)],
        'Ms1den2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',14)],
        'Ms0dan2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',16)],
        'Ms2den2':L[idx_find(L, 'İLK YARI/MAÇ SONUCU',18)],
        
        'Alt15':L[idx_find(L, '1,5 ALT/ÜST',2)],
        'Ust15':L[idx_find(L, '1,5 ALT/ÜST',4)],
        'Alt25':L[idx_find(L, '2,5 ALT/ÜST',2)],
        'Ust25':L[idx_find(L, '2,5 ALT/ÜST',4)],
        'Alt35':L[idx_find(L, '3,5 ALT/ÜST',2)],
        'Ust35':L[idx_find(L, '3,5 ALT/ÜST',4)],
        'Alt45':L[idx_find(L, '4,5 ALT/ÜST',2)],
        'Ust45':L[idx_find(L, '4,5 ALT/ÜST',4)],
        
        'MS1VE15A':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',2)],
        'MS0VE15A':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',4)],
        'MS2VE15A':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',6)],
        'MS1VE15U':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',8)],
        'MS0VE15U':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',10)],
        'MS2VE15U':L[idx_find(L, 'MS VE 1,5 ALT/ÜST',12)],
        'MS1VE25A':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',2)],
        'MS0VE25A':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',4)],
        'MS2VE25A':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',6)],
        'MS1VE25U':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',8)],
        'MS0VE25U':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',10)],
        'MS2VE25U':L[idx_find(L, 'MS VE 2,5 ALT/ÜST',12)],
        'MS1VE35A':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',2)],
        'MS0VE35A':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',4)],
        'MS2VE35A':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',6)],
        'MS1VE35U':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',8)],
        'MS0VE35U':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',10)],
        'MS2VE35U':L[idx_find(L, 'MS VE 3,5 ALT/ÜST',12)],
        ### kalan satirlari sor ve ona gore yap
        'IYAlt05':L[idx_find(L, '1. YARI DEPLASMAN 0,5 ALT/ÜST',2)],
        'IYUst05':L[idx_find(L, '1. YARI DEPLASMAN 0,5 ALT/ÜST',4)],
        #####
        'IYAlt25':L[idx_find(L, 'EV SAHIBI 2,5 A/Ü',2)],
        'IYUst25':L[idx_find(L, 'EV SAHIBI 2,5 A/Ü',4)],
        #####
        'IYAlt15':L[idx_find(L, 'EV SAHIBI 1,5 A/Ü',2)],
        'IYUst15':L[idx_find(L, 'EV SAHIBI 1,5 A/Ü',4)],
        
        ####EV SAHIBI 0,5 A/Ü
        'Ev05Alt':L[idx_find(L, 'EV SAHIBI 0,5 A/Ü',2)],
        'Ev05Ust':L[idx_find(L, 'EV SAHIBI 0,5 A/Ü',4)],
        ####EN ÇOK GOL OLACAK YARI
        'EnCokGol1Yari':L[idx_find(L, 'EN ÇOK GOL OLACAK YARI',2)],
        'EnCokGolEsit':L[idx_find(L, 'EN ÇOK GOL OLACAK YARI',4)],
        'EnCokGol2Yari':L[idx_find(L, 'EN ÇOK GOL OLACAK YARI',6)],
        ###DEPLASMAN 2,5 A/Ü
        'Dep25Alt':L[idx_find(L, 'DEPLASMAN 2,5 A/Ü',2)],
        'Dep25Ust':L[idx_find(L, 'DEPLASMAN 2,5 A/Ü',4)],
        #####
        'Dep05Alt':L[idx_find(L, 'DEPLASMAN 0,5 A/Ü',2)],
        'Dep05Ust':L[idx_find(L, 'DEPLASMAN 0,5 A/Ü',4)],
        ###DEPLASMAN 1,5 A/Ü
        'Dep15Alt':L[idx_find(L, 'DEPLASMAN 1,5 A/Ü',2)],
        'Dep15Ust':L[idx_find(L, 'DEPLASMAN 1,5 A/Ü',4)],
        ####EV SAHIBI İKI YARIYI DA KAZANIR
        'Ms1den1':L[idx_find(L, 'EV SAHIBI İKI YARIYI DA KAZANIR',2)],
        'ES2YKH':L[idx_find(L, 'EV SAHIBI İKI YARIYI DA KAZANIR',4)],
        ####1. YARI EV SAHIBI 0,5 ALT/ÜST
        'IYEv05Alt':L[idx_find(L, '1. YARI EV SAHIBI 0,5 ALT/ÜST',2)],
        'IYEv05Ust':L[idx_find(L, '1. YARI EV SAHIBI 0,5 ALT/ÜST',4)],
        ####DEPLASMAN İKI YARIYI DA KAZANIR
        'D2YKE':L[idx_find(L, 'DEPLASMAN İKI YARIYI DA KAZANIR',2)],
        'D2YKH':L[idx_find(L, 'DEPLASMAN İKI YARIYI DA KAZANIR',4)],
        
        ##bazi maclarin idaa oranlarinda kisaltma kodlari yok!!!!
        '1YSONEVSAH':L[idx_find(L, '1. YARI SONUCU',2)],
        '1YSONBER':L[idx_find(L, '1. YARI SONUCU',4)],
        '1YSONDEP':L[idx_find(L, '1. YARI SONUCU',6)],
        #TEK/ÇIFT
        'TEK':L[idx_find(L, 'TEK/ÇIFT',2)],
        'CIFT':L[idx_find(L, 'TEK/ÇIFT',4)],
        #KARŞILIKLI GOL
        'KARGOLVAR':L[idx_find(L, 'KARŞILIKLI GOL',2)],
        'KARGOLYOK':L[idx_find(L, 'KARŞILIKLI GOL',4)],
        
        
        'runTime':datetime.now().strftime("%d/%m/%Y")
        
    
        
        
    }
    # delete of fake last Nan value (-1)
    L.pop()
    
    df = pd.DataFrame([cleaned])
    return df

In [45]:
L=['Al Baten', 'Al Qaisoma', '2', 'MS', '3', '18-12-23 | 1. Lig | 14. Hafta | Stat: Al-Batin FC Stadium']



ikyarisonucu=find_IYScore(L)
print(ikyarisonucu)

-
